In [1]:
import os
os.chdir('../')
os.getcwd()

'/Users/dennis/repos/study/mt/mt_rdf'

In [2]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import time

In [3]:
LIMIT = False
def parse_to_sparql(query):
    if not LIMIT:
        return f'SELECT * WHERE {{ {query} }}'
    return f'SELECT * WHERE {{ {query} }} LIMIT {LIMIT}'

In [4]:
def execute_sparql(query):
    """This function executes the sparql query and returns the results"""
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    # sparql.setQuery('PREFIX :  <http://unics.cloud/ontology/>\nPREFIX onto: <http://www.ontotext.com/>' + query)
    sparql.setQuery(query)
    # the previous query as a literal string
    try:
        start_time = time.time()
        results = sparql.query()
        execution_time = time.time() - start_time

        results = results.convert()
        return results, execution_time
    #     clean_results = []
    #     for row in results['results']['bindings']:
    #         interim_results=[]
    #         for i in range(len(results['head']['vars'])):
    #             if 'datatype' in row.get(results['head']['vars'][i]).keys():
    #                 value = int(row.get(results['head']['vars'][i])['value'])
    #                 interim_results.append(value)
    #             else:
    #                 value = row.get(results['head']['vars'][i])['value'] 
    #                 interim_results.append(value)
    #         if len(interim_results) != 0:
    #             clean_results.append(tuple(interim_results))
    #     return clean_results, execution_time
    except Exception as e:
        print(e)
    return None, 0

In [5]:
df_multiple_bgps = pd.read_csv('data/queries/wdbench/multiple_bgps.txt', header=None)
df_multiple_bgps.rename(columns={0: 'id', 1: 'query_parts'}, inplace=True)

In [6]:
df_multiple_bgps

,id,query_parts
0,1,<http://www.wikidata.org/entity/Q8337> <http:/...
1,2,<http://www.wikidata.org/entity/Q896916> <http...
2,3,?x1 <http://www.wikidata.org/prop/direct/P1010...
3,4,?x1 <http://www.wikidata.org/prop/direct/P1010...
4,5,?x1 <http://www.wikidata.org/prop/direct/P1015...
...,...,...
676,677,?x1 ?x2 <http://www.wikidata.org/entity/Q31> ....
677,678,?x1 ?x2 <http://www.wikidata.org/entity/Q48487...
678,679,?x1 ?x2 <http://www.wikidata.org/entity/Q5> . ...
679,680,?x1 ?x2 <http://www.wikidata.org/entity/Q65364...


In [7]:
df_multiple_bgps.iloc[0]['query_parts']

'<http://www.wikidata.org/entity/Q8337> <http://www.wikidata.org/prop/direct/P50> ?x1 . <http://www.wikidata.org/entity/Q8337> <http://www.wikidata.org/prop/direct/P674> ?x2 . '

In [8]:
parse_to_sparql(df_multiple_bgps.iloc[0]['query_parts'])

'SELECT * WHERE { <http://www.wikidata.org/entity/Q8337> <http://www.wikidata.org/prop/direct/P50> ?x1 . <http://www.wikidata.org/entity/Q8337> <http://www.wikidata.org/prop/direct/P674> ?x2 .  }'

In [9]:
execute_sparql(parse_to_sparql(df_multiple_bgps.iloc[0]['query_parts']))

({'head': {'vars': ['x1', 'x2']},
  'results': {'bindings': [{'x1': {'type': 'uri',
      'value': 'http://www.wikidata.org/entity/Q34660'},
     'x2': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q173998'}},
    {'x1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34660'},
     'x2': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q174009'}},
    {'x1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34660'},
     'x2': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q176132'}},
    {'x1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34660'},
     'x2': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q176772'}},
    {'x1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34660'},
     'x2': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q187923'}},
    {'x1': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q34660'},
     'x2': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q

In [10]:
results = pd.DataFrame(columns=['query_id', 'exec_time_in_s'])

In [14]:
path = 'results/wdbench/'
def res_to_logs(result: dict):
    res_df = pd.DataFrame(result)
    print(res_df)
    res_df.to_csv(path + 'multiple_bgps_results.csv')



In [16]:
res_dict = {'query_id': [], 'exec_time': []}
for index, row in df_multiple_bgps.iterrows():
    query = parse_to_sparql(row['query_parts'])
    _, time_s = execute_sparql(query=query)
    res_dict['query_id'].append(row['id'])
    res_dict['exec_time'].append(time_s)
    print(index)
    # if index % 13 == 0:
    res_to_logs(result=res_dict)

0
   query_id  exec_time
0         1   0.075071
1
   query_id  exec_time
0         1   0.075071
1         2   0.431131
2
   query_id  exec_time
0         1   0.075071
1         2   0.431131
2         3   0.607868


KeyboardInterrupt: 